In [ ]:
# Run this only once.
# import aws_s3_data_handler
# csv_filenames = aws_s3_data_handler.download_zip_and_load_to_destination_bucket()

In [ ]:
csv_filenames = ['syhdr_commercial_inpatient_2016.csv',
 'syhdr_commercial_outpatient_2016.csv',
 'syhdr_commercial_person_2016.csv',
 'syhdr_commercial_pharmacy_2016.csv',
 'syhdr_medicaid_inpatient_2016.csv',
 'syhdr_medicaid_outpatient_2016.csv',
 'syhdr_medicaid_person_2016.csv',
 'syhdr_medicaid_pharmacy_2016.csv',
 'syhdr_medicaid_provider_2016.csv',
 'syhdr_medicare_inpatient_2016.csv',
 'syhdr_medicare_outpatient_2016.csv',
 'syhdr_medicare_person_2016.csv',
 'syhdr_medicare_pharmacy_2016.csv',
 'syhdr_medicare_provider_2016.csv']

In [ ]:
import aws_s3_data_handler
csv_paths = aws_s3_data_handler.return_csv_files(csv_filenames)

In [ ]:
import data_handler
import database_handler
db_session = database_handler.create_connection()
dict_list = list()
for csv_path in csv_paths:
    pandas_df = data_handler.return_pandas_df_from_csv(csv_path)
    table_name = csv_path.split('/')[3].split('.')[0].replace('syhdr_','').replace('_2016','')
    stmtnt = data_handler.return_create_statement_from_df(pandas_df,'sydh_data',table_name)
    database_handler.execute_query(db_session, stmtnt)
    stmtnt = data_handler.return_truncate_statement('sydh_data',table_name)
    database_handler.execute_query(db_session, stmtnt)
    stmtnt = data_handler.return_insert_statement_from_df(pandas_df,'sydh_data',table_name)
    database_handler.execute_query(db_session, stmtnt)
    dict_item = {}
    dict_item['table_name'] = table_name
    dict_item['columns'] =  pandas_df.columns.tolist()
    dict_item['sample_data'] = pandas_df.head(2)
    dict_list.append(dict_item)
database_handler.close_connection(db_session)

In [ ]:
import pandas as pd
summary_df = pd.DataFrame(dict_list)
summary_df.to_csv('summary.csv')

In [1]:
import pandas as pd
summary_df =pd.read_csv('summary.csv')

In [2]:
import data_handler
table_data = data_handler.return_df_as_table_data(summary_df)

In [3]:
import openai_handler
response_data = openai_handler.open_ai_run_analysis(table_data, "Give me all the tables that are commercial")

In [4]:
response = response_data.choices[0].message.content

In [5]:
print(response)

To define the relationship between the two tables `commercial_inpatient` and `commercial_outpatient`, we will analyze their structures and how they interact in the context of a healthcare database.

### Table Overview:

1. **commercial_inpatient**:
   - This table represents inpatient claims for patients receiving care in a hospital or healthcare facility. 
   - Key identifiers include:
     - **PERSON_ID**: Unique identifier for each patient.
     - **FACILITY_ID**: Identifier for the healthcare facility providing the service.
     - **CLM_CNTL_NUM**: Claim control number to track the specific claim.
   - Contains numerous clinical and billing details such as admission type, diagnosis codes, procedure codes, and financial amounts.

2. **commercial_outpatient**:
   - This table represents outpatient claims for patients receiving care but not admitted to a facility.
   - Key identifiers include:
     - **PERSON_ID**: Unique identifier for each patient (the same as in the inpatient table